# Libs


In [1]:
!pip install -q openai
!pip install -q transformers
!pip install -q PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 693.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import openai
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import pandas as pd
import re
import re
import PyPDF2
import nltk
import string
import nltk
nltk.download('punkt')
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import pipeline
from tqdm import tqdm
import json

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data pre-processing:
####As the task of interest is the generate quality data from the book, we need to split the book into meaningful and coherent parts with the consideration of max token limitation of the model.
*   For achieving this first I extracted chapter titles together with numbers for subsections.

*  Then text of subsections were extracted.

*  It is splitted from a point in which there is a title within a subsection. This way, the document coherency won't be interrupted and generated data will be meaningful and good quality.

* Due to model token limitations (2048), it is simply splitted in half after applying the steps above.

In [4]:
def extract_toc(pdf_path, toc_page_numbers):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        toc_data = {}
        current_title = None
        for page_number in toc_page_numbers:
            page = reader.pages[page_number - 1]
            page_text = page.extract_text()

            lines = page_text.strip().split('\n')
            for line in lines:
                line = line.strip()


                subtitle_match = re.match(r"^(\d+\.\d+)", line)
                if subtitle_match and current_title is not None:
                    subtitle_number = subtitle_match.group(1)
                    toc_data[current_title].append(subtitle_number)
                else:

                    title_match = re.match(r"^\d [^\.]+", line)
                    if title_match:
                        title = line.split(' ', maxsplit=1)[1]
                        toc_data[title] = []
                        current_title = title

    return toc_data


In [5]:
def extract_chapter_texts(pdf_path):
    toc_page_numbers = [5, 6, 7, 8]
    toc_titles_with_numbers = extract_toc(pdf_path, toc_page_numbers)

    toc_list = [{"title": key, "subtitles": value} for key, value in toc_titles_with_numbers.items()]

    chapter_texts = []
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        for toc_data in toc_list:
            chapter_title = toc_data["title"]
            subtitle_numbers = toc_data["subtitles"]

            chapter_start_page_match = re.search(r'\d+', chapter_title)
            chapter_start_page = int(chapter_start_page_match.group()) if chapter_start_page_match else 1

            chapter_end_page_match = re.search(r'\d+', toc_list[toc_list.index(toc_data) + 1]["title"]) if toc_list.index(toc_data) < len(toc_list) - 1 else None
            chapter_end_page = int(chapter_end_page_match.group()) + 1 if chapter_end_page_match else num_pages - 1

            chapter_text = ""
            for page_number in range(chapter_start_page, chapter_end_page):
                page = reader.pages[page_number - 1]
                page_text = page.extract_text()
                chapter_text += page_text

            cleaned_chapter_text = chapter_text.replace('\n', ' ').strip()

            if chapter_title not in toc_titles_with_numbers.keys():
                chapter_dict = {'chapter_name': chapter_title, 'chapter_text': cleaned_chapter_text}
                chapter_texts.append(chapter_dict)

            for subtitle_number in subtitle_numbers:
                subtitle_start = cleaned_chapter_text.find(subtitle_number)
                subtitle_end = (
                    cleaned_chapter_text.find(subtitle_numbers[subtitle_numbers.index(subtitle_number) + 1])
                    if subtitle_numbers.index(subtitle_number) < len(subtitle_numbers) - 1
                    else len(cleaned_chapter_text)
                )
                subtitle_text = cleaned_chapter_text[subtitle_start:subtitle_end].strip()

                subtitle_key = f"{chapter_title} {subtitle_number}"
                if subtitle_key not in toc_titles_with_numbers.keys():
                    subtitle_dict = {
                        'chapter_name': subtitle_key,
                        'chapter_text': subtitle_text
                    }
                    chapter_texts.append(subtitle_dict)

    return chapter_texts


def remove_dicts_with_empty_text(chapters_list):
    return [chapter_dict for chapter_dict in chapters_list if any(c.isalnum() for c in chapter_dict['chapter_text'])]


In [6]:
#for splitting from a a new title point within a given chapter
def count_chapter_text(chapters_list):
    new_chapters_list = []
    for chapter_dict in chapters_list:
        chapter_title = chapter_dict['chapter_name']
        chapter_text = chapter_dict['chapter_text']
        tokens = nltk.word_tokenize(chapter_text)
        if len(tokens) > 3000:
            lines = chapter_text.split('.')
            split_point = None
            for i in range(len(lines)):
                line = lines[i]
                if re.match(r'^([A-Z][^.!?]*[^.!?] ?)+$', line):
                    split_point = i
                    break
            if split_point:

                first_part_text = '\n'.join(lines[:split_point])
                second_part_title = lines[split_point]
                second_part_dict = {
                    'chapter_name': second_part_title,
                    'chapter_text': '\n'.join(lines[split_point + 1:])
                }


                first_part_dict = {
                    'chapter_name': chapter_title,
                    'chapter_text': first_part_text
            }
                new_chapters_list.append(first_part_dict)
                new_chapters_list.append(second_part_dict)

            else:
                new_chapters_list.append(chapter_dict)

        else:
            new_chapters_list.append(chapter_dict)

    return new_chapters_list


In [7]:
def split_text_into_halves(chapters):
    def split_text(text):
        sentences = nltk.sent_tokenize(text)
        mid_index = len(sentences) // 2
        first_half = " ".join(sentences[:mid_index])
        second_half = " ".join(sentences[mid_index:])
        return first_half, second_half

    result = []
    for d in chapters:
          first_half, second_half = split_text(d['chapter_text'])
          result.append((d['chapter_name'],first_half))
          result.append((d['chapter_name'],second_half))
    return result


In [8]:
pdf_path = './data/rust-doc.pdf'
chapters = extract_chapter_texts(pdf_path)
del chapters[0] #as it is authors note
chapters=remove_dicts_with_empty_text(chapters)
docs=split_text_into_halves(chapters) #each chapter splitted into 2 due to max token limitations.

In [9]:
len(docs)

154

# MODEL
#### For the task of interest, I decided to use Falcon-7B Instruct as it is finetuned on a mixture of chat/instruct datasets which makes it a suitable opensource model for our task.
- Since the model requires memory more than one available in Colab, I used quantization to load the model in 4 bit.

In [10]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [11]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)


tokenizer = AutoTokenizer.from_pretrained(model_id)

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
pipeline = transformers.pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_new_tokens=2500,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id)


#Toy example
#### After trying several prompt templates, I found the following template most efficient.

In [ ]:
x,y=split_text_into_halves(chapters[2]["chapter_text"])

In [ ]:
prompt = "Read the following document about Rust Programming Language and create a Q&A problem where the question is a coding problem and the answer is the code."
prompt_template = f"### Instruction: {prompt}{y}\n### Response:\nQuestion:"

In [ ]:
sequences = pipeline(prompt_template)

In [ ]:
print(seq['generated_text'])

### Instruction: Read the following document about Rust Programming Language and create a Q&A problem where the question is a coding problem and the answer is the code.Check out the src/main.rs file: Filename: src/main.rs fnmain(){ println! ("Hello, world! "); } Now let’s compile this “Hello, world!” program and run it in the same step using the cargo run command: $ cargo run Compiling guessing_game v0.1.0 (file:///projects/guessing_ game) Running `target/debug/guessing_game` Hello, world! Therun command comes in handy when you need to rapidly iterate on a project, and this game is such a project: we want to quickly test each iteration before moving on to the next one. Reopen the src/main.rs file. Y ou’ll be writing all the code in this file.
### Response:
Question: How can we modify the existing "Hello, world!" program to take two parameters?
Answer: We can modify the program by adding more code that allows us to use two inputs (in this case, two strings) and modify the output based o

# With all Documents:
#### Due to computational limitations (i.e., GPU), I was able to use half of the documents I created from the book.

In [14]:
def get_qa(docs):
  responses=[]
  for doc in tqdm(docs):
    prompt = "Read the following document about Rust Programming Language and create a Q&A problem where the question is a coding problem and the answer is the code."
    prompt_template = f"### Instruction: {prompt}{doc[1]}\n### Response:\nQuestion:"
    sequences = pipeline(prompt_template)
    for seq in sequences:
      response= seq['generated_text']
      responses.append((doc[0],response))

  return responses


In [18]:
responses=get_qa(docs[:60])

  0%|          | 0/60 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 60/60 [28:43<00:00, 28.73s/it]


# RESULTS

In [58]:
responses[0]

('Introduction 9 1.2',
 '### Instruction: Read the following document about Rust Programming Language and create a Q&A problem where the question is a coding problem and the answer is the code.1.2 Installation The first step to using Rust is to install it. Y ou’ll need an internet con- nection to run the commands in this chapter, as we’ll be downloading Rust from the internet. W e’ll be showing off a number of commands using a terminal, and those lines all start with $. Y ou don’t need to type in the $character; they are there to indicate the start of each command. Y ou’ll see many tutorials and examples around the web that follow this convention: $ for commands run as a regular user, and #for commands you should be running as an administrator. Lines that don’t start with $are typically showing the output of the previous command. Installing on Linux or Mac If you’re on Linux or a Mac, all you need to do is open a terminal and type this: $ curl https://sh.rustup.rs -sSf | sh This will d

In [29]:
file_path = '/content/drive/MyDrive/falcon_responses.json'
with open(file_path, 'w') as json_file:
    json.dump(responses, json_file)

In [36]:
def get_response_section(text):
    start_marker = "Response:\n"
    start_index = text.find(start_marker)

    if start_index != -1:
        start_index += len(start_marker)
        response_section = text[start_index:]
        return response_section.strip()
    else:
        return None

In [42]:
l=[]
for output in responses:
  text=get_response_section(output[1])
  title=output[0]
  l.append((title,text))


In [49]:
l[7]

('Guessing Game 21 2.2',
 'Question: What should we do when there are two possible answers, and the user types something completely wrong? In the first case, we would be able to print out an error message like this: `You got {} out of 2!`. In the second case, it would be better to simply return an `Err`.\n\nAnswer: We should return an `Err` when the user types something completely wrong. This will make the rest- ery process easier and less prone to errors. We could also use a try-catch block to catch the `Err`, but the `Ok` and `Err` variants are more intuitive in this case.\n\nExplanation: We should use the `Err` type for errors in the program, which are defined using the `|| operator`. This operator means “or” and can be used to define an error type. To print an error message and handle the value at the same time, we should use the `||` operator, which means that the program will only execute if the user inputs something correct. If there is no input, we can simply print an error. Th

In [50]:
file_path = '/content/drive/MyDrive/falcon_Q&As.json'
with open(file_path, 'w') as json_file:
    json.dump(l, json_file)

In [51]:
with open(file_path, 'r') as json_file:
    data = json.load(json_file)


[['Introduction 9 1.2', 'Question: What is the first command needed to use Rust?\nA. rustup\nB. rustup install\nC. rustup init\nD. rustup -c\nE. rustup run'], ['Introduction 9 1.2', 'Question: What version of the Rust programming language should I use as my default shell?\nA: If you have reasons for preferring not to use rustup.rs, please see the Rust installation page for other options.\nB: Once you have Rust installed, updating to the latest version is easy \nC: Uninstalling is as easy as installing it.\nD: If you install Rust successfully, you can see the version number, commit hash, and commit date in a format similar to this for the latest stable version at the time you install.'], ['Introduction 9 1.3', 'Question: 2.4.3'], ['Introduction 9 1.3', 'Question: Are you interested in learning about the different programming languages? Answer: Yes.\nIf the answer is yes, I highly recommend learning about the following languages: • C++ •.NET • Java • Node.js • JavaScript • C# • Swift • T

In [56]:
df = pd.DataFrame(data, columns=['Section', 'Output'])

df.head(10)

,Section,Output
0,Introduction 9 1.2,Question: What is the first command needed to ...
1,Introduction 9 1.2,Question: What version of the Rust programming...
2,Introduction 9 1.3,Question: 2.4.3
3,Introduction 9 1.3,Question: Are you interested in learning about...
4,Guessing Game 21 2.1,Question: \nWhat type of data structure is use...
5,Guessing Game 21 2.1,Question: Create a Q&A problem with a coding p...
6,Guessing Game 21 2.2,Question: You will need to modify the followin...
7,Guessing Game 21 2.2,Question: What should we do when there are two...
8,Guessing Game 21 2.3,Question: 2.3 Generating a Secret Number \n\n2...
9,Guessing Game 21 2.3,"Question: (0.3.14, rand::Rng) How do I generat..."


In [57]:
df.to_csv("/content/drive/MyDrive/falcon_Q&As.csv", index=False)